In [103]:
import pandas as pd 
from IPython.display import clear_output, display, HTML


combined_data = pd.read_csv(f'combined_data_remaining.csv')
#combined_data = pd.read_csv(f'combined_data.csv')
salary_data = pd.read_csv(f'./clean_salary_data.csv')
new_combine = pd.read_csv(f'new_combined_data.csv')


#left_only.head()
majors_df = pd.read_csv('code_departments.csv')
#print(majors_df.index[majors_df['Major'] == "CPMS"].to_list()[0])
#print(majors_df.at[majors_df.index[majors_df['Major'] == "CPMS"].to_list()[0], "Department"])
#print(majors_df.index[majors_df['Major'] == "FGSM"].to_list())

def find_similar(first, last, courses, outer, year):
    c = []
    for i, course in enumerate(courses.replace("'", "")[1:-1].split(", ")):
        index = majors_df.index[majors_df['Major'] == course[:4]].to_list()
        if index != []:
            temp = majors_df.at[index[0], "Department"]
            if len(temp) == 4:
                c.append(temp)
            elif temp == 'Public Policy':
                c.append('PLCY')
    #print(c)
    names = []
    if len(first) > 1:
        names.append(first)
    if len(last) > 1:
        names.append(last)
    if "-" in last:
        names.append(last.split("-")[0])
        names.append(last.split("-")[1])
    #print(names)

    outer["first_name"] = outer["name"].apply(lambda x: x.split(" ")[0])
    outer["last_name"] = outer["name"].apply(lambda x: x.split(" ")[1])


    df = outer.loc[(outer["name"]!=first+" "+last)&
                   (outer['years_taught'].apply(lambda x: year in x))&
                   ((outer["first_name"].isin(names)) |
                    (outer["real_name(s)"].apply(lambda x: any(substring in x for substring in names))) |
                    (outer["last_name"].isin(names))
                    )][["real_name(s)", "years_taught", "departments","salaries", "name"]]
    df = df[df['departments'].str.contains('|'.join(c))]
    df = df.sort_values('real_name(s)', ascending=True)
    bad_dep = ["VPSA", "VPAF", "SVPA", "PRES", "VPAA", "LIBR", "VPA-", "DIT-", 
               "VPR-", "IT-N", "VPUR", "IT-I"]
    for i, row in df.iterrows():
        for dep in bad_dep:
            if dep in row["departments"]:
                df.drop(i, inplace = True)
                break
    return df.reset_index().drop("index", axis = 1)

responses = ["","n","never", "no", "nah", "nar", "nope", "poop"]

count = 0
combined_data["years_taught"] = combined_data["years_taught"].apply(lambda x: x.replace("'"," ")[1:-1].split(', '))
for i, row in combined_data.head(30).iterrows():
    semesters = row['semester'].replace("'"," ")[1:-1].split(', ')
    replaced = False
    for sem in semesters:
        year = int(sem[-5:].replace(" ",""))
        if "fall" in sem: 
            year += 1
        year = str(year)
        if int(year) >= 2013 and int(year) <= 2022 and (not year in row["years_taught"]) and (not replaced):
            print(f"missing year {year}")
            replaced = True
            first = row["name"].split(" ")[0]
            last = row["name"].split(" ")[1]
            #print(first, last)
            count +=1
            similar = find_similar(first,last, row["course"], salary_data, year)
            if len(similar) >= 1:
                display(HTML(similar.to_html()))
                response = input(f"{first} {last}: ")
                if (not response in responses):
                    new_years = similar.loc[int(response), 'years_taught'][1:-1].replace(" "," ").split(",")
                    combined_data.loc[i, 'years_taught'].extend(new_years)

                    new_salaries = similar.loc[int(response), 'salaries'][1:-1].replace(" "," ").split(",")
                    combined_data.loc[i, 'salaries'].extend(new_salaries)

                    new_departments = similar.loc[int(response), 'departments'][1:-1].replace(" "," ").split(",")
                    combined_data.loc[i, 'departments'].extend(new_departments)

                    new_combine = new_combine.append(combined_data.loc[i], ignore_index=True)

                    combined_data.drop(i, inplace = True)
                else:
                    combined_data.drop(i, inplace = True)
                #row["years_taught"] = row["years_taught"].extend(similar.loc[int(response), "years_taught"])
                clear_output()

print(count)
combined_data.to_csv(f'combined_data_remaining.csv', index=False)
new_combine.to_csv('new_combined_data.csv', index = False)

missing year 2020
missing year 2013


,real_name(s),years_taught,departments,salaries,name
0,"Binkley, Adam Ross","[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]","['ARHU', 'ARHU', 'USG-', 'USG-', 'USG-', 'USG-', 'USG-', 'USG-', 'USG-', 'USG-']","[16317.960000000001, 25762.06, 45900.0, 45900.0, 54492.48, 54492.48, 55582.33, 58111.41, 59273.64, 61363.04]",ADAM BINKLEY


KeyboardInterrupt: Interrupted by user